# Introduktion til Prompt Engineering
Prompt engineering er processen med at designe og optimere prompts til opgaver inden for naturlig sprogbehandling. Det handler om at vælge de rigtige prompts, justere deres parametre og vurdere deres præstation. Prompt engineering er afgørende for at opnå høj nøjagtighed og effektivitet i NLP-modeller. I dette afsnit vil vi gennemgå det grundlæggende i prompt engineering ved hjælp af OpenAI-modeller til udforskning.


### Øvelse 1: Tokenisering
Undersøg tokenisering med tiktoken, en open source og hurtig tokenizer fra OpenAI
Se [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) for flere eksempler.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Øvelse 2: Bekræft opsætning af Github Models-nøgle

Kør koden nedenfor for at sikre, at din Github Models-endpoint er sat korrekt op. Koden tester blot en simpel prompt og tjekker, om der kommer et svar. Inputtet `oh say can you see` bør blive fulgt op af noget i stil med `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Øvelse 3: Opdigtede svar
Undersøg hvad der sker, når du beder LLM’en om at give svar på en prompt om et emne, der måske ikke eksisterer, eller om emner, den måske ikke kender til, fordi de ligger uden for dens fortrænede datasæt (mere nylige emner). Se hvordan svaret ændrer sig, hvis du prøver en anden prompt eller en anden model.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Øvelse 4: Instruktionsbaseret
Brug variablen "text" til at angive det primære indhold
og variablen "prompt" til at give en instruktion, der relaterer sig til det primære indhold.

Her beder vi modellen om at opsummere teksten for en elev i 2. klasse


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Øvelse 5: Kompleks prompt
Prøv en forespørgsel, der har system-, bruger- og assistentbeskeder
Systemet sætter assistentens kontekst
Bruger- og assistentbeskeder giver kontekst til en samtale over flere omgange

Bemærk, hvordan assistentens personlighed er sat til "sarkastisk" i systemkonteksten.
Prøv at bruge en anden personlighedskontekst. Eller prøv en anden række af input/output-beskeder


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Øvelse: Udforsk din intuition
Eksemplerne ovenfor giver dig mønstre, du kan bruge til at lave nye prompts (simple, komplekse, instruktioner osv.) – prøv at lave andre øvelser for at undersøge nogle af de andre idéer, vi har talt om, som eksempler, hints og mere.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiske oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel, menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå ved brug af denne oversættelse.
